In [111]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import joblib
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from catboost import Pool, CatBoostClassifier
from kaggler.model import AutoLGB
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss
import random

# 1. 문제 정의

# 2. 데이터 수집

## (1) 데이콘 기본 데이터

In [13]:
train = pd.read_csv('data/train.csv').drop(['index'], axis=1).fillna('NAN')
test = pd.read_csv('data/test.csv').drop(['index'], axis=1).fillna('NAN')
sample_submission = pd.read_csv('data/sample_submission.csv')

# 4. 탐색적 데이터 분석

# 5. 변수 조정

## (1) 이상치 제거

In [14]:
merge_data = pd.concat([train, test], axis = 0)

# DAYS_BIRTH
merge_data['DAYS_BIRTH_month']=np.floor((-merge_data['DAYS_BIRTH'])/30)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/30)/12).astype(int)*12)
merge_data['DAYS_BIRTH_week']=np.floor((-merge_data['DAYS_BIRTH'])/7)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/7)/4).astype(int)*4)

# DAYS_EMPLOYED
merge_data['DAYS_EMPLOYED_month']=np.floor((-merge_data['DAYS_EMPLOYED'])/30)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['DAYS_EMPLOYED_week']=np.floor((-merge_data['DAYS_EMPLOYED'])/7)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/7)/4).astype(int)*4)

# before_EMPLOYED
merge_data['before_EMPLOYED']=merge_data['DAYS_BIRTH']-merge_data['DAYS_EMPLOYED']
merge_data['before_EMPLOYED_month']=np.floor((-merge_data['before_EMPLOYED'])/30)-(
    (np.floor((-merge_data['before_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['before_EMPLOYED_week']=np.floor((-merge_data['before_EMPLOYED'])/7)-(
    (np.floor((-merge_data['before_EMPLOYED'])/7)/4).astype(int)*4)

# DAYS_BIRTH
merge_data['1new_1'] = merge_data['DAYS_BIRTH_month'] / merge_data['income_total']
merge_data['2new_1'] = merge_data['DAYS_BIRTH_week'] / merge_data['income_total']

# DAYS_EMPLOYED
merge_data['10new_1'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['income_total']
merge_data['11new_1'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['income_total']

# before_EMPLOYED
merge_data['12new_1'] = merge_data['before_EMPLOYED'] / merge_data['income_total']
merge_data['13new_1'] = merge_data['before_EMPLOYED_month'] / merge_data['income_total']
merge_data['14new_1'] = merge_data['before_EMPLOYED_week'] / merge_data['income_total']

# 융합 삭제
#merge_data['3new_1'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['DAYS_BIRTH_month']
#merge_data['4new_1'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['DAYS_BIRTH_week']
#merge_data['5new_1'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['DAYS_BIRTH_month']
#merge_data['6new_1'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['DAYS_BIRTH_week']

#merge_data['7new_1'] =  merge_data['begin_month'] / merge_data['DAYS_BIRTH_month']
#merge_data['8new_1'] =  merge_data['begin_month'] / merge_data['DAYS_EMPLOYED_month']
#merge_data['9new_1'] =  merge_data['begin_month'] / merge_data['before_EMPLOYED_month']


# 소득대비 
merge_data['DAYS_BIRTH'] = merge_data['DAYS_BIRTH'] / -365
merge_data['DAYS_EMPLOYED'] = merge_data['DAYS_EMPLOYED'] / -365

merge_data['new_1'] = merge_data['child_num'] / merge_data['income_total']
merge_data['new_2'] = merge_data['family_size'] / merge_data['income_total']
merge_data['new_3'] = merge_data['DAYS_BIRTH'] / merge_data['income_total']
merge_data['new_4'] = merge_data['DAYS_EMPLOYED'] / merge_data['income_total']
merge_data['new_5'] = merge_data['begin_month'] / merge_data['income_total']
merge_data['new_6'] =  merge_data['DAYS_EMPLOYED'] / merge_data['DAYS_BIRTH']

# 소득 skewed-data 처리
# merge_data['log_income_total'] = np.log(merge_data['income_total'])
# merge_data['sqrt_income_total'] = np.sqrt(merge_data['income_total'])
# merge_data['boxcox_income_total'] = stats.boxcox(merge_data['income_total'])[0]

merge_data = merge_data.fillna(-999)
train = merge_data[merge_data['credit'] != -999]
test = merge_data[merge_data['credit'] == -999]
test.drop('credit', axis = 1, inplace = True)

train = train[train['child_num']<=6].reset_index(drop=True) # 아이의 수가 7명 이상인 데이터 제거

## 인코딩

In [15]:
train_oh = train.copy()
train_noh = train.copy()
test_oh = test.copy()
test_noh = test.copy()

In [16]:
object_col = []
for col in train_noh.columns:
    if train_noh[col].dtype == 'object':
        train_noh[col] = train_noh[col].astype('category')
        test_noh[col] = test_noh[col].astype('category')

In [17]:
object_col = []
for col in train_oh.columns:
    if train_oh[col].dtype == 'object':
        object_col.append(col)
print(object_col)        
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train_oh.drop(object_col, axis=1, inplace=True)
train_oh = pd.concat([train_oh, train_onehot_df], axis=1)    

test_onehot_df = pd.DataFrame(enc.transform(test_oh.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test_oh.drop(object_col, axis=1, inplace=True)
test_oh = pd.concat([test_oh, test_onehot_df], axis=1)

['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type']


In [18]:
true = train[['credit']]
true['0'] = true['credit'][true['credit']==0.0]
true['1'] = true['credit'][true['credit']==1.0]
true['2'] = true['credit'][true['credit']==2.0]
del true['credit']
true = true.replace([0.0, 2.0], [1.0, 1.0])
true = true.fillna(0)
true = true.values

# 6. 모델 학습

In [19]:
pred_dict = {}
pred_test_dict = {}

## (1) Lightgbm

In [20]:
train_x = train_noh.drop(['credit'], axis=1)
train_y = train_noh['credit']
test_x = test_noh.copy()

### Parameter Tuning

### 3 seeds x 5 folds

In [21]:
lucky_seeds=[42,2019,91373]
for i, seed in enumerate(lucky_seeds):

    kfold = KFold(n_splits=10, random_state = seed, shuffle = True)
    cv=np.zeros((train.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train)):

        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

        lgbmodel = LGBMClassifier(learning_rate=0.005, objective='multiclass', n_estimators=10000, num_leaves=1000, 
                                  max_depth=-1, min_child_weight=2, colsample_bytree=0.4,  
                                   n_jobs=-1, random_state=seed)

        lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=30, verbose=None) 
        #joblib.dump(lgbmodel, f'./pred_pkl/LGB_{n+1}_fold_{seed}_seed_lgb.pkl')

        # CROSS-VALIDATION , EVALUATE CV
        cv[val_idx,:] = lgbmodel.predict_proba(x_val)
        pred_test += lgbmodel.predict_proba(test_x) / 10
    pred_dict['lgb'+str(i+1)] = cv
    pred_test_dict['lgb'+str(i+1)] = pred_test
        
    print('multi_logloss :', log_loss(true, cv))

multi_logloss : 0.6915101725662152
multi_logloss : 0.6915843058850291
multi_logloss : 0.6912776413331292


lgbmodels_path = os.listdir('./pred_pkl/')
lgbmodels_list = [x for x in lgbmodels_path if x.endswith("lgb.pkl")]
assert len(lgbmodels_list) == 15
lgb_preds = np.zeros((test_x.shape[0], 3))

for m in lgbmodels_list:
    lgbmodel = joblib.load('./pred_pkl/'+m)
    lgb_preds_proba = lgbmodel.predict_proba(test)
    lgb_preds += lgb_preds_proba/15

## (2) XGBoost

In [21]:
train_x = train_oh.drop(['credit'], axis=1)
train_y = train_oh['credit']
test_x = test_oh.copy()

### Parameter Tuning (hyperopt)

In [141]:
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.1, random_state=0)

In [144]:
#-------------------------------------------------#
SEED=0
# Scoring and optimization functions


def score(params):
    print("Training with params: ")
    print(params)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
    gbm_model = xgb.train(params, dtrain, 100000,
                          evals=watchlist,
                          early_stopping_rounds=30,
                          verbose_eval=100000)
    predictions = gbm_model.predict(dvalid,
                                    ntree_limit=gbm_model.best_iteration + 1)
    score = log_loss(y_test, predictions)
    # TODO: Add the importance for the selected features
    print("\tlog_loss {0}\n\n".format(log_loss))
    # The score function should return the loss (1-score)
    # since the optimize function looks for the minimum
    return {'loss': score, 'status': STATUS_OK}


def optimize(random_state=SEED):
    """
    This is the optimization function that given a space (space here) of 
    hyperparameters and a scoring function (score here), finds the best hyperparameters.
    """
    # To learn more about XGBoost parameters, head to this page: 
    # https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
    param = {'objective':'multi:softprob', 'seed':seed, 'num_class': 3, 'eval_metric':'mlogloss', 
         'eta': 0.01, 'min_child_weight': 3,
         'colsample_bytree': 0.3, 'colsample_bylevel': 0.6, 'subsample': 0.8
        }
    space = {
        'eta': hp.quniform('eta', 0.004, 0.01, 0.001),
        # A problem with max_depth casted to float instead of int with
        # the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(20, 70, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.3, 1, 0.05),
        'colsample_bylevel': hp.quniform('colsample_bylevel', 0.3, 1, 0.05),
        'eval_metric': 'mlogloss',
        'objective' : 'multi:softprob',
        # Increase this number if you have more cores. Otherwise, remove it and it will default 
        # to the maxium number. 
        'num_class' : 3,
        'seed': random_state
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest, 
                # trials=trials, 
                max_evals=3)
    return best

#-------------------------------------------------#


# Load processed data

# You could use the following script to generate a well-processed train and test data sets:
# https://www.kaggle.com/yassinealouini/predicting-red-hat-business-value/features-processing
# I have only used the .head() of the data sets since the process takes a long time to run.
# I have also put the act_train and act_test data sets since I don't have the processed data sets 
# loaded. 



#-------------------------------------------------#

# Run the optimization

# Trials object where the history of search will be stored
# For the time being, there is a bug with the following version of hyperopt.
# You can read the error messag on the log file.
# For the curious, you can read more about it here: https://github.com/hyperopt/hyperopt/issues/234
# => So I am commenting it.
# trials = Trials()

best_hyperparams = optimize()
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

Training with params:                                
{'colsample_bylevel': 0.65, 'colsample_bytree': 0.35000000000000003, 'eta': 0.008, 'eval_metric': 'mlogloss', 'gamma': 0.75, 'max_depth': 65, 'min_child_weight': 4.0, 'num_class': 3, 'objective': 'multi:softprob', 'seed': 0, 'subsample': 0.8500000000000001}
[0]	train-mlogloss:1.09368	valid-mlogloss:1.09467    

[646]	train-mlogloss:0.34869	valid-mlogloss:0.68941  

	log_loss <function log_loss at 0x7ff390cbe9d0>      


Training with params:                                                           
{'colsample_bylevel': 0.8500000000000001, 'colsample_bytree': 0.45, 'eta': 0.005, 'eval_metric': 'mlogloss', 'gamma': 0.9, 'max_depth': 36, 'min_child_weight': 2.0, 'num_class': 3, 'objective': 'multi:softprob', 'seed': 0, 'subsample': 0.8}
[0]	train-mlogloss:1.09492	valid-mlogloss:1.09593                               

[866]	train-mlogloss:0.33132	valid-mlogloss:0.69113                             

	log_loss <function log_loss at 0x7f

In [23]:
y_test = pd.DataFrame(y_test, columns=['credit'])
y_test['0'] = y_test['credit'][y_test['credit']==0.0]
y_test['1'] = y_test['credit'][y_test['credit']==1.0]
y_test['2'] = y_test['credit'][y_test['credit']==2.0]
del y_test['credit']
y_test = y_test.replace([0.0, 2.0], [1.0, 1.0])
y_test = y_test.fillna(0)
y_test = y_test.values

In [24]:
y_train = pd.DataFrame(y_train, columns=['credit'])
y_train['0'] = y_train['credit'][y_train['credit']==0.0]
y_train['1'] = y_train['credit'][y_train['credit']==1.0]
y_train['2'] = y_train['credit'][y_train['credit']==2.0]
del y_train['credit']
y_train = y_train.replace([0.0, 2.0], [1.0, 1.0])
y_train = y_train.fillna(0)
y_train = y_train.values

true = train[['credit']]
true['0'] = true['credit'][true['credit']==0.0]
true['1'] = true['credit'][true['credit']==1.0]
true['2'] = true['credit'][true['credit']==2.0]
del true['credit']
true = true.replace([0.0, 2.0], [1.0, 1.0])
true = true.fillna(0)
true = true.values

### Parameter Tuning (GridSearchCV)

In [52]:
%%time
params = {'learning_rate':[0.02, 0.01, 0.006],
          'max_depth': [30] # 튜닝할 파라미터 삽입
            }
xgb_clf = XGBClassifier(n_estimators=100, min_child_weight=2, 
                        colsample_bytree=0.8, colsample_bylevel=0.8, subsample=0.8, tree_method='gpu_hist',
                        num_class=3, objective='multiclass', n_jobs=-1)

grid_cv = GridSearchCV(xgb_clf, param_grid=params, cv=3, n_jobs=-1, verbose=300)
grid_cv.fit(train_x, train_y)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done   2 out of   9 | elapsed:    5.9s remaining:   21.0s
[Parallel(n_jobs=-1)]: Done   3 out of   9 | elapsed:    5.9s remaining:   12.0s
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed: 26.4min remaining: 33.0min
[Parallel(n_jobs=-1)]: Done   5 out of   9 | elapsed: 32.3min remaining: 25.8min
[Parallel(n_jobs=-1)]: Done   6 out of   9 | elapsed: 33.0min remaining: 16.5min
[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed: 39.0min remaining: 11.1min
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 44.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 44.4min finished
[10:03:31] DEBUG: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_gpu_hist.cu:899: [GPU Hist]: Configure
[10:03:31] ======== Monitor: 

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=0.8,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=2,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=-1, num_class=3,
                                     num_parallel_tree=None,
                                     objective='multiclass', random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, sub

In [53]:
hr_grid_df = pd.DataFrame(grid_cv.cv_results_)
hr_grid_df.loc[:, ['mean_test_score', "params"]]

,mean_test_score,params
0,0.715538,"{'learning_rate': 0.02, 'max_depth': 30}"
1,0.716824,"{'learning_rate': 0.01, 'max_depth': 30}"
2,NaN,"{'learning_rate': 0.006, 'max_depth': 30}"


In [50]:
%%time
params = {'learning_rate':[0.03, 0.01],
          'max_depth': [18, 23] # 튜닝할 파라미터 삽입
            }
xgb_clf = XGBClassifier(n_estimators=100, min_child_weight=2, 
                        colsample_bytree=0.8, colsample_bylevel=0.8, subsample=0.8, tree_method='gpu_hist',
                        num_class=3, objective='multiclass', n_jobs=-1)

grid_cv = GridSearchCV(xgb_clf, param_grid=params, cv=3, n_jobs=-1, verbose=300)
grid_cv.fit(train_x, train_y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:  8.4min remaining: 42.0min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:  8.5min remaining: 25.4min
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:  8.7min remaining: 17.5min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:  8.8min remaining: 12.3min
[Parallel(n_jobs=-1)]: Done   6 out of  12 | elapsed:  8.8min remaining:  8.8min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed: 10.0min remaining:  7.1min
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed: 10.0min remaining:  5.0min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed: 10.0min remaining:  3.3min
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed: 10.1min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 10.2min remaining:

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=0.8,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=2,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=-1, num_class=3,
                                     num_parallel_tree=None,
                                     objective='multiclass', random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, sub

In [51]:
hr_grid_df = pd.DataFrame(grid_cv.cv_results_)
hr_grid_df.loc[:, ['mean_test_score', "params"]]

,mean_test_score,params
0,0.716370,"{'learning_rate': 0.03, 'max_depth': 18}"
1,0.714669,"{'learning_rate': 0.03, 'max_depth': 23}"
2,0.714178,"{'learning_rate': 0.01, 'max_depth': 18}"
3,0.715463,"{'learning_rate': 0.01, 'max_depth': 23}"


### 3 seeds x 5 folds

In [110]:
lucky_seeds=[42, 2019, 91373]
xgtest = xgb.DMatrix(test_x)
for i, seed in enumerate(lucky_seeds):

    kfold = KFold(n_splits=10, random_state = seed, shuffle = True)
    cv = np.zeros((train.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)

    for n, (train_idx, val_idx) in enumerate(kfold.split(train)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        
        dtrain = xgb.DMatrix(x_train, label=y_train)
        dvalid = xgb.DMatrix(x_val, label=y_val)
        watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

        param = {'objective':'multi:softprob', 'seed':seed, 'num_class': 3, 'eval_metric':'mlogloss', 
                 'eta': 0.01, 'min_child_weight': 3,
                 'colsample_bytree': 0.3, 'colsample_bylevel': 0.6, 'subsample': 0.8
                }

        xgbmodel = xgb.train(param, dtrain, 100000, watchlist, early_stopping_rounds=30, verbose_eval=100)
        #joblib.dump(xgbmodel, f'./pred_pkl/XGB_{n+1}_fold_{seed}_seed_xgb.pkl')

        cv[val_idx, :] = xgbmodel.predict(dvalid)
        pred_test += xgbmodel.predict(xgtest) / 10
        
    pred_dict['xgb'+str(i+1)] = cv
    pred_test_dict['xgb'+str(i+1)] = pred_test
    print('multi_logloss:', log_loss(true, cv))

[0]	train-mlogloss:1.09467	valid-mlogloss:1.09470
[100]	train-mlogloss:0.87353	valid-mlogloss:0.87954
[200]	train-mlogloss:0.80531	valid-mlogloss:0.82073
[300]	train-mlogloss:0.77498	valid-mlogloss:0.79960
[400]	train-mlogloss:0.75462	valid-mlogloss:0.78835
[500]	train-mlogloss:0.73858	valid-mlogloss:0.78039
[600]	train-mlogloss:0.72476	valid-mlogloss:0.77373
[700]	train-mlogloss:0.71219	valid-mlogloss:0.76847
[800]	train-mlogloss:0.70067	valid-mlogloss:0.76373


KeyboardInterrupt: 

xgbmodels_path = os.listdir('./pred_pkl/')
xgbmodels_list = [x for x in xgbmodels_path if x.endswith("xgb.pkl")]
assert len(xgbmodels_list) == 15
xgb_preds = np.zeros((test_x.shape[0], 3))
xgtest = xgb.DMatrix(test_X)

for m in xgbmodels_list:
    xgbmodel = joblib.load('./pred_pkl/'+m)
    xgb_preds_proba = xgbmodel.predict_proba(xgtest)
    xgb_preds += xgb_preds_proba/15

## (3) Random Forest

### GridSearchCV

params = {'max_depth': [55, 60, 65] # 튜닝할 파라미터 삽입
            }

rf_clf = RandomForestClassifier(random_state = 0, n_estimators = 1000, 
                                min_samples_leaf=2, min_samples_split=2,
                                criterion='entropy', n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 5, n_jobs = -1)
grid_cv.fit(df_train, y)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

### 3 seeds, 5 folds

In [ ]:
lucky_seeds=[42,2019,91373]
for i, seed in enumerate(lucky_seeds):

    kfold = KFold(n_splits=10, random_state = seed, shuffle = True)
    cv = np.zeros((train.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

        rfmodel = RandomForestClassifier(n_estimators=1000, criterion='entropy', max_depth=55,
                                         min_samples_leaf=2, min_samples_split=2,
                                         random_state=seed)
        rfmodel.fit(x_train, y_train)
        #joblib.dump(rfmodel, f'./pred_pkl/RF_{n+1}_fold_{seed}_seed_rf.pkl')
        
        cv[val_idx, :] = rfmodel.predict_proba(x_val)        
        pred_test += rfmodel.predict_proba(test_x) / 10
        
    pred_dict['rf'+str(i+1)] = cv
    pred_test_dict['rf'+str(i+1)] = pred_test
    print('multi_logloss :', log_loss(true, cv))

rfmodels_path = os.listdir('./pred_pkl/')
rfmodels_list = [x for x in rfmodels_path if x.endswith("rf.pkl")]
assert len(rfmodels_list) == 15
rf_preds = np.zeros((test_x.shape[0], 3))

for m in rfmodels_list:
    rfmodel = joblib.load('./pred_pkl/'+m)
    rf_preds_proba = rfmodel.predict_proba(test_x)
    rf_preds += rf_preds_proba/15

## (4) Catboost (성능X)

lucky_seeds=[42,2019,91373]
for i, seed in enumerate(lucky_seeds):

    kfold = KFold(n_splits=5, random_state = seed, shuffle = True)
    cv = np.zeros((train.shape[0], 3))
    pred_test = np.zeros((test_x.shape[0], 3), dtype=float)
    
    for n, (train_idx, val_idx) in enumerate(kfold.split(train)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]
        _train = Pool(x_train, label=y_train)
        _valid = Pool(x_val, label=y_val)

        catmodel =  CatBoostClassifier(loss_function='MultiClass', early_stopping_rounds=50, 
                                       random_state=seed, learning_rate=0.02, iterations=100000
                                       #task_type="GPU"
                                      )
        
        catmodel.fit(_train, eval_set=_valid, use_best_model=True, verbose=2000)
        #joblib.dump(rfmodel, f'./pred_pkl/RF_{n+1}_fold_{seed}_seed_rf.pkl')
        
        cv[val_idx, :] = catmodel.predict_proba(x_val)        
        pred_test += catmodel.predict_proba(test_x) / 5
        
    pred_dict['cat'+str(i+1)] = cv
    pred_test_dict['cat'+str(i+1)] = pred_test
    print('multi_logloss :', log_loss(true, cv))

## (4) Stacking (AutoLGB)

### 27features = 3seed(42, 2019, 91373) x 3model(lgb, xgb, rf) x 3class(0, 1, 2)

In [ ]:
X_train = pd.DataFrame(np.hstack([x for _, x in pred_dict.items()]))
X_test = pd.DataFrame(np.hstack([x for _, x in pred_test_dict.items()]))

pred = np.zeros((X_train.shape[0], 3), dtype=float)
pred_test = np.zeros((X_test.shape[0], 3), dtype=float)
#kfold = KFold(n_splits=5, random_state = seed, shuffle = True)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i_cv, (i_trn, i_val) in enumerate(cv.split(X_train, train_y)):
    if i_cv == 0:
        clf = AutoLGB(objective='multiclass', metric='multi_logloss', params={'num_class': 3}, 
                      feature_selection=False, n_est=10000)
        clf.tune(X_train.iloc[i_trn], train_y[i_trn])
        n_best = clf.n_best
        features = clf.features
        params = clf.params
        print(f'best iteration: {n_best}')
        print(f'selected features ({len(features)}): {features}')        
        print(params)
        clf.fit(X_train.iloc[i_trn], train_y[i_trn])
    else:
        train_data = lgb.Dataset(X_train[features].iloc[i_trn], label=train_y[i_trn])
        clf = lgb.train(params, train_data, n_best, verbose_eval=100)
    
    pred[i_val] = clf.predict(X_train[features].iloc[i_val])
    pred_test += clf.predict(X_test[features]) / 5

In [ ]:
print(f'CV Log Loss: {log_loss(train_y, pred):.6f}')

# 결과 제출

In [ ]:
submission = sample_submission.copy()
submission.iloc[:, 1:] = pred_test

In [ ]:
submission.to_csv('submission/submission.csv', index=False)